In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential, load_model
import numpy as np
import tensorflow as tf
from glob import glob

In [ ]:
tf.debugging.set_log_device_placement(True)

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices(device_type='GPU'))

In [ ]:
dataset_path = './dataset'

In [ ]:
train_dataset_path = dataset_path + '/train'
valid_dataset_path = dataset_path + '/test'

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in inception.layers:
  layer.trainable = False

In [ ]:
folders = glob(train_dataset_path + '/*')
print(folders)

In [ ]:
x = Flatten()(inception.output)

prediction = Dense(len(folders), activation='sigmoid')(x)

model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
x

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dataset = train_data_gen.flow_from_directory(train_dataset_path,
                                                   target_size=(224, 224),
                                                   batch_size=64,
                                                   class_mode='categorical')

test_dataset = test_data_gen.flow_from_directory(valid_dataset_path,
                                                   target_size=(224, 224),
                                                   batch_size=64,
                                                   class_mode='categorical')

In [ ]:
r = model.fit(train_dataset,
              validation_data=test_dataset,
              epochs=5,
              steps_per_epoch=len(train_dataset),
              validation_steps=len(test_dataset))

In [ ]:
model.save('./models/my_model.h5')

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
import os

In [ ]:
def load_image(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

In [ ]:
def_dir = './dataset/test/def_front'
ok_dir = './dataset/test/ok_front'

In [ ]:
model = load_model('./models/my_model.h5')

In [ ]:
def_dir_list = os.listdir(def_dir)
def_dir_list

In [ ]:
result = []
for file in def_dir_list:
    image = load_image(def_dir + '/' + file)
    pred = model.predict(image)
    result.append(np.argmax(pred))

In [ ]:
print(f'True Defect : {result.count(0)} | False Defect : {result.count(1)} | Accuracy : {result.count(0)/(result.count(0) + result.count(1)) * 100}%')

In [ ]:
ok_dir_list = os.listdir(ok_dir)
ok_dir_list

In [ ]:
result = []
for file in ok_dir_list:
    image = load_image(ok_dir + '/' + file)
    pred = model.predict(image)
    result.append(np.argmax(pred))

In [ ]:
print(f'True OK : {result.count(1)} | False OK : {result.count(0)} | Accuracy : {result.count(1)/(result.count(0) + result.count(1)) * 100}%')